### Importing libraries

In [1]:
import numpy as np
import pandas as pd
import math

### Importing data

In [2]:
data1=pd.read_csv(r'interpolation_of_ships_trial_3.csv')
data2=pd.read_csv(r'sumo results ROI.csv')

In [3]:
def removing_false_alarms(data):
    h=[]
    for i in range(len(data)):
        if type(data['vds_target__boat__false_alarm_cause'][i])==str:
            h.append(i)
    
    data1=data.drop(h,axis=0).reset_index()
    
    return data1
    

In [4]:
data2 = removing_false_alarms(data2)
x = data1
y = data2

In [5]:
def haversine(lat1, lon1, lat2, lon2): 
      
    # distance between latitudesand longitudes 
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0
  
    # convert to radians 
    lat1 = (lat1) * math.pi / 180.0
    lat2 = (lat2) * math.pi / 180.0
  
    # apply haversine formulae 
    a = (pow(math.sin(dLat / 2), 2) + 
         pow(math.sin(dLon / 2), 2) * 
             math.cos(lat1) * math.cos(lat2)); 
    rad = 6371000
    c = 2 * math.asin(math.sqrt(a)) 
    return rad * c 

def Diff(li1, li2): 
    # Calculating difference between 2 sets
    return (list(set(li1) - set(li2)))

In [9]:
class sar_ais_neighbourhood:
    
    def __init__(self):
        print('SAR - AIS Neighbourhood class is initiated.')
    
    def find_neighbours(self, x, y, len_threshold, len_perc_threshold, dis_threshold):
        '''
        x - contains AIS data
        y - contains SUMO data
        Note that schema of both dataframes should be set as expected.
        
        len_threshold (in meters) - Length error between SAR data and AIS data that is allowed to match vessels
        len_perc_threshold - Percentage of length error between SAR data and AIS data that is allowed to match vessels
        dis_threshold (in meters) - Distance error that is allowed between SAR ship location and AIS ship location to match vessels
        '''
        nei1=[]
        nei2=[]
        e=[]
        f=[]
        nei=[]
        d=[]

        for k in range(10):
            n1=Diff(range(len(x)),nei1)
            n2=Diff(range(len(y)),nei2)
            dis=np.array([])
            idx=[]
            o=0
            for u in n1:
                lat1 = x['LATITUDE'][u]
                lon1 = x['LONGITUDE'][u]
                a=np.array([])  
                for j in n2:
                    lat2 = y['vds_target__boat__lat'][j]
                    lon2 = y['vds_target__boat__lon'][j]
                    a=np.append(a,haversine(lat1, lon1,lat2, lon2)) 
                    idx.insert(o,(u,j))
                    o=o+1
                dis=np.append(dis,a)

            z=[x for _,x in sorted(zip(dis,idx))]
            q=sorted(dis)

            for i in range(len(z)):
                if (((abs(x['LENGTH'][z[i][0]]-y['vds_target__boat__length'][z[i][1]])<len_threshold)|
                    (abs(x['LENGTH'][z[i][0]]-y['vds_target__boat__length'][z[i][1]])/x['LENGTH'][z[i][0]]<len_perc_threshold))&(q[i]<dis_threshold)):
                    if ((z[i][0] not in e)&(z[i][1] not in f)):
                        e.append(z[i][0])
                        f.append(z[i][1])
                        nei1.append(z[i][0])
                        nei2.append(z[i][1])
                        nei.append((z[i][0],z[i][1]))
                        d.append(q[i])
                        
        return nei, d


In [10]:
integrated=pd.DataFrame()
neighbourhood_model = sar_ais_neighbourhood()

nei, d = neighbourhood_model.find_neighbours(x,y,30,0.2,1000)

SAR - AIS Neighbourhood class is initiated.


C:\ProgramData\anaconda3\envs\CNN\lib\site-packages\ipykernel_launcher.py:46: RuntimeWarning: divide by zero encountered in longlong_scalars


### Storing integrated data into dataframe

In [11]:
for i in range(len(nei)):
    a=nei[i][0]
    b=nei[i][1]
    m=pd.DataFrame([[x['MMSI'][a],y['vds_target__boat__lat'][b],y['vds_target__boat__lon'][b],x['LATITUDE'][a],x['LONGITUDE'][a],
                    d[i],y['vds_target__boat__xpixel'][b],y['vds_target__boat__ypixel'][b],y['vds_target__boat__length'][b],
                     x['LENGTH'][a],y['vds_target__boat__width'][b],x['BREADTH'][a],y['vds_target__boat__inc_ang'][b],x['NAME'][a],x['AISTYPE'][a],
                   y['vds_target__boat__nr_pixels'][b], float(y['vds_target__boat__max_value'][b].split(',')[2]),
                   float(y['vds_target__boat__significance'][b].split(',')[2])]],
                   columns=['MMSI','Detected_lat','Detected_lon','AIS_lat','AIS_lon','Distance_error','xpixel','ypixel',
                                               'LENGTH_detected','LENGTH','BREADTH_detected','BREADTH','incidence_angle','NAME','AISTYPE','Total Pixel','Max_value','Significance'])
    
    integrated=pd.concat([integrated,m],axis=0,ignore_index=True,sort=False)
    

#### SUMO detected latitudes and longitudes are provided with AIS detected latitudes and longitudes. Length errors and distance errors are given for comparison of matched ships.

In [12]:
integrated

,MMSI,Detected_lat,Detected_lon,AIS_lat,AIS_lon,Distance_error,xpixel,ypixel,LENGTH_detected,LENGTH,BREADTH_detected,BREADTH,incidence_angle,NAME,AISTYPE,Total Pixel,Max_value,Significance
0,341611000,18.924825,72.866922,18.924800,72.866800,13.130130,8882,11947,85,110,66,14,35.953,S CAS,52,59,297.0,8.351
1,419104000,18.956207,72.859282,18.956300,72.859200,13.464897,9027,11627,96,82,43,14,36.031,NAFISA I,70,44,964.0,30.349
2,419001201,18.924717,72.864094,18.924696,72.863960,14.275850,8911,11954,99,80,63,18,35.968,WILCHIEF 1,39,63,987.0,31.710
3,419093100,18.951980,72.858420,18.951900,72.858300,15.440130,9027,11674,56,32,41,9,36.031,MV CHANDRA NEPTUNE,32,26,654.0,20.047
4,419000882,18.895360,72.843614,18.895500,72.843600,15.636808,9061,12308,191,183,87,32,36.049,OAKTREE,80,173,1425.0,49.788
5,419078500,18.947727,72.863069,18.947600,72.863000,15.877154,8970,11710,69,57,47,12,36.000,OFFSHORE HUNTER,90,36,797.0,23.920
6,419058400,18.941906,72.857720,18.941800,72.857600,17.268777,9013,11783,55,38,32,9,36.023,MV KAMRUP,52,20,322.0,9.013
7,419000147,18.962744,72.858776,18.962600,72.858700,17.895852,9046,11558,46,25,39,8,36.041,SURYA SHAKTI,52,18,454.0,13.833
8,419001364,18.931621,72.866465,18.931527,72.866320,18.514292,8901,11876,87,67,55,15,35.963,PRIYA TWENTY THREE,90,53,896.0,28.286
9,419001117,18.928260,72.868201,18.928166,72.868049,19.069793,8876,11908,86,88,53,18,35.950,TAG 20,70,49,570.0,17.437


In [13]:
integrated.to_csv('Integrated_23Oct2018.csv',index=False)